In [2]:
import os
import uuid
import shutil
from copy import deepcopy
import tempfile
from pathlib import Path
from PIL import Image, ImageOps
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import png
import json
import numpy as np
from matplotlib import pyplot as plt  # necessary import for PIL typing # noqa: F401
from typing import Tuple, List, Dict, Union, Optional

In [17]:
dicom_file_path = '/Users/xbkaishui/Desktop/J002_original.dcm'
ds = pydicom.dcmread(dicom_file_path)
ds
# TDDO hack part
# ds.SamplesPerPixel = 1
# ds.pixel_array
# ds.pixel_array

array([[19105, 19044, 19055, ..., 19862, 19877, 19885],
       [19165, 18961, 19007, ..., 19887, 19881, 19866],
       [19138, 19199, 19208, ..., 19855, 19862, 19895],
       ...,
       [ 1736,  1714,  1752, ...,  1779,  1770,  1795],
       [ 1725,  1743,  1776, ...,  1795,  1811,  1732],
       [ 1707,  1714,  1767, ...,  1795,  1810,  1841]], dtype=uint16)

In [9]:
def _check_if_greyscale(instance: pydicom.dataset.FileDataset) -> bool:
        """Check if a DICOM image is in greyscale.

        :param instance: A single DICOM instance.

        :return: FALSE if the Photometric Interpretation is RGB.
        """
        # Check if image is grayscale using the Photometric Interpretation element
        try:
            color_scale = instance.PhotometricInterpretation
        except AttributeError:
            color_scale = None
        is_greyscale = color_scale in ["MONOCHROME1", "MONOCHROME2"]

        return is_greyscale

In [10]:
def _rescale_dcm_pixel_array(
        instance: pydicom.dataset.FileDataset, is_greyscale: bool
    ) -> np.ndarray:
        """Rescale DICOM pixel_array.

        :param instance: A singe DICOM instance.
        :param is_greyscale: FALSE if the Photometric Interpretation is RGB.

        :return: Rescaled DICOM pixel_array.
        """
        # Normalize contrast
        if "WindowWidth" in instance:
            if is_greyscale:
                image_2d = apply_voi_lut(instance.pixel_array, instance)
            else:
                image_2d = instance.pixel_array
        else:
            image_2d = instance.pixel_array

        # Convert to float to avoid overflow or underflow losses.
        image_2d_float = image_2d.astype(float)

        if not is_greyscale:
            image_2d_scaled = image_2d_float
        else:
            # Rescaling grey scale between 0-255
            image_2d_scaled = (
                np.maximum(image_2d_float, 0) / image_2d_float.max()
            ) * 255.0

        # Convert to uint
        image_2d_scaled = np.uint8(image_2d_scaled)

        return image_2d_scaled

In [12]:
is_greyscale = _check_if_greyscale(ds)

        # Rescale pixel array
image = _rescale_dcm_pixel_array(ds, is_greyscale)
shape = image.shape
shape

(3008, 1586)

In [14]:
def save_pixel_array_as_png(
        pixel_array: np.array,
        is_greyscale: bool,
        output_file_name: str = "example",
        output_dir: str = "temp_dir",
    ) -> None:
        """Save the pixel data from a loaded DICOM instance as PNG.

        :param pixel_array: Pixel data from the instance.
        :param is_greyscale: True if image is greyscale.
        :param output_file_name: Name of output file (no file extension).
        :param output_dir: String path to output directory.
        """
        shape = pixel_array.shape

        # Write the PNG file
        os.makedirs(output_dir, exist_ok=True)
        if is_greyscale:
            with open(f"{output_dir}/{output_file_name}.png", "wb") as png_file:
                w = png.Writer(shape[1], shape[0], greyscale=True)
                w.write(png_file, pixel_array)
        else:
            with open(f"{output_dir}/{output_file_name}.png", "wb") as png_file:
                w = png.Writer(shape[1], shape[0], greyscale=False)
                # Semi-flatten the pixel array to RGB representation in 2D
                pixel_array = np.reshape(pixel_array, (shape[0], shape[1] * 3))
                w.write(png_file, pixel_array)

        return None

In [15]:
save_pixel_array_as_png(image, is_greyscale, "test", "/tmp/")